In [1]:
import os
os.chdir('/workspace')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import torch
from dataset.dataset_iter import data_formatting
warnings.filterwarnings('ignore')
from tqdm import tqdm

device="cuda"
d_type=torch.float32

In [2]:
import os

# 현재 위치
os.getcwd()

'/workspace'

# Modeling

In [3]:
def Mamba2Model():
    from model.mamba_stock import Mambamodeling
    pretrained_path="/workspace/weight/Mamba2"
    model=Mambamodeling(
            d_model=768,
            d_inermediate=2048,
            n_layer=8,
            layer="Mamba2",
            pretrained_path=pretrained_path
        ).to(device)
    model.eval()
    return model


def Mamba1Model():
    from model.mamba_stock import Mambamodeling
    pretrained_path="/workspace/weight/Mamba1"
    model=Mambamodeling(
            d_model=768,
            d_inermediate=2048,
            n_layer=8,
            layer="Mamba1",
            pretrained_path=pretrained_path
        ).to(device)
    model.eval()
    return model

def TransformerModel():
    from model.qwen_stock import QwenStock
    from transformers import AutoConfig
    
    model_name = "Qwen/Qwen2.5-Coder-0.5B-Instruct"
    
    qwenconfig=AutoConfig.from_pretrained(model_name,
                    intermediate_size= 2048,
                    max_position_embeddings= 128,
                    num_hidden_layers=8,
                    vocab_size= 5
                    )

    model=QwenStock(qwenconfig).from_pretrained("/workspace/weight/0000085000").to(device)
    return model

def predict(data,model,seq_length):
    stock_predictions=[]
    total= len(data)-seq_length
    
    for start_idx in tqdm(range(total), desc="Prediction", ncols=80, ascii=True):
        open=data.loc[data.index[start_idx], "Open"]
        input=data_formatting(data,start_idx,1,seq_length,d_type)
        with torch.no_grad():
            pred=model(input)[0,-1,:]
        price_pred=pred[3]*open/100
        stock_predictions.append(price_pred)
        if start_idx%200==0:
            torch.cuda.empty_cache()
    stock_predictions=torch.stack(stock_predictions, dim=0).cpu().numpy()
    data["predict"]=np.concatenate((np.zeros(seq_length), stock_predictions))
    return data


# Prediction

In [ ]:
from backtesting.test import EURUSD,GOOG

data_path="/workspace/data/HBARUSDT_hour.csv"
stock=pd.read_csv(data_path)
stock=EURUSD

model=TransformerModel()
predict(stock,model,127)

Prediction:  22%|#####7                    | 1084/4873 [00:07<00:25, 147.57it/s]

In [ ]:
# 데이터를 DataFrame으로 변환
#time= pd.to_datetime(stock['open_time'], unit='ms')
time=stock.index
# data2와 data1을 그래프로 그리기
plt.figure(figsize=(15, 10))  # 그래프 크기 설정 (선택적)
plt.plot(time, stock['Close'], label='real')
plt.plot(time, stock["predict"], label='predict')

# 그래프에 레이블과 범례 추가
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Real and Predict Over Time')
plt.legend()

# 그래프 표시
plt.show()

In [ ]:
from backtesting import Strategy, Backtest
from backtesting.lib import SignalStrategy

class System(Strategy):
    def init(self):
        super().init()
        
    def next(self):
        price = self.data.Close[-1]
        
        if (self.data.predict - price)/price > 0 and (self.data.predict - price)/price<0.1 and self.data.predict != 0:
            if not self.position.is_long:
                self.buy()
        else:
            self.position.close()
       
            
backtest = Backtest(stock, System,
              cash=10000,
              #commission=0.0002,
              exclusive_orders=True)

backtest.run()

In [ ]:
backtest.plot()